In [256]:
import advent
import numpy as np
import numpy.typing as npt

# Preprocessing utility function
data = advent.get_lines(9, map_fn = lambda line: np.array(list(int(n) for n in line.split(" "))))

In [257]:
def continuation(line: npt.NDArray[np.int64]) -> int:
    if np.sum(np.abs(line)) == 0: return 0
    diff = np.diff(line)
    return line[-1] + continuation(diff)

sum([continuation(line) for line in data])

1938800261

In [258]:
def continuation_left(line: npt.NDArray[np.int64]) -> int:
    if np.sum(np.abs(line)) == 0: return 0
    diff = np.diff(line)
    return line[0] - continuation_left(diff) # it cant be this easy, right?

sum([continuation_left(line) for line in data])

1112

In [ ]:
# OPTIONAL PART, NOT PART OF EXERCISE
# I wanted to actually recover the polynomial
# With some help of the Mathologer :)

In [259]:
import math

def npdiff(line: list[int]):
    return [line[ix] - line[ix-1] for ix in range(1, len(line))]

def npsub(a: list[int], b: list[int]) -> list[int]:
    assert len(a) == len(b)
    return [a[i] - b[i] for i in range(len(a))]

def get_col(line: list[int]) -> list[int]:
    if len(line) == 1: return line
    diff = npdiff(line)
    colr = get_col(diff)
    return [line[0]] + colr

def lim(pow: int, mul: int) -> list[int]:
    seq: list[int] = [mul*x**pow for x in range(pow+1)]
    return get_col(seq)

def recover_col(col: list[int]) -> list[int]:
    # Recovers polynomial. if col is len n, assume its polynomial degree n-2
    degree = len(col) - 1
    if degree == 0: return col # constant value, degree 0
    # The multiplier of degree=degree is mul
    mul = col[-1] // math.factorial(degree)
    # now subtract the chained differences of mul*n**pow
    #print(f"col: {col}, lim: {lim(degree, mul)}, degree: {degree}, mul: {mul}")
    new = npsub(col, lim(degree, mul))[:-1]
    return recover_col(new) + [mul]

def recover(line: list[int]) -> list[int]:#
    return recover_col(get_col(line))
    

In [260]:
# Kinda ugly but I didn't really care at this point, just wanted to finish it
def pretty_print(factors: list[int]):
    result = ""
    for ix in range(len(factors)):
        if factors[ix] == 0: continue
        elif factors[ix] == 1 and ix == 1: result += "x"
        elif factors[ix] == 1 and ix > 1: result += f"x^{ix}"
        elif factors[ix] == -1 and ix == 1: result += "-x"
        elif factors[ix] == -1 and ix > 1: result += f"-x^{ix}"
        elif ix == 0: result += f"{factors[0]}"
        elif ix == 1: result += f"{factors[1]}x"
        else: result += f"{factors[ix]}x^{ix}"
        result += " + "
    print(result[:-3])

pretty_print([0, 1, 3])

x + 3x^2


In [261]:
def full(input: list[int]):
    pretty_print(recover(input))

full([100, 2, 8, 4, -9, 1])

100 + -236x + 203x^2 + -78x^3 + 14x^4 + -x^5
